In [ ]:
import requests
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+str(appid), params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()

In [ ]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
        'json' : 1,
        'filter' : 'all',
        'language' : 'koreana',#한국어로된 리뷰
        'day_range' : 9223372036854775807,#전체기간의 리뷰
        'review_type' : 'all',
    }
    while n > 0:
        time.sleep(2)
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100
        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        if len(response['reviews']) < 100: break
    return reviews

In [ ]:
df_id=pd.read_csv('1. appid 크롤링 결과.csv') #appid  불러오기
len(df_id)

In [ ]:
for i in range(99,9899,100):
    print(i)
    df = pd.DataFrame()
    reviews = []
    game_name=[] 
    for index,row in tqdm(df_id[i-99:i].iterrows()):
        reviews.append(get_n_reviews(row[1], 100))
        game_name.append(row[0])
    for j in range(len(reviews)):
      data = pd.DataFrame.from_dict(reviews[j])
      data['game_name'] = game_name[j]
      df = df.append(data)

In [ ]:
df.reset_index(drop = True)
df = df[['game_name','review','steam_purchase']]
df['review'] = df['review'] \
.replace(r'[^가-힣 ]', ' ', regex=True) \
.replace("'", '') \
.replace(r'\s+', ' ', regex=True) \
.str.strip() \
.str[:255]
df = df[df['review'].str.strip().astype(bool)]

In [ ]:
df.to_csv('2. review 크롤링 결과.csv',index=False)